In [71]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from mixture_of_experts import MoE
import pandas as pd
import numpy as np
from utils import get_data_per_anchor
from sklearn.utils import shuffle
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from micromlgen import port

In [72]:
df = pd.read_pickle("mid_concrete.pkl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
features = ['power', 'pdda_input_real_1', 'pdda_input_real_2', 'pdda_input_imag_2', 'pdda_input_real_3', 'pdda_input_imag_3', 'pdda_input_real_4', 'pdda_input_imag_4',
           'pdda_input_real_5', 'pdda_input_imag_5']
# theta does not change, make predictions for phi only, azimuth angles 
# predict only for phi, 
targets = ['true_phi']
location = ['x_tag', 'y_tag']
pdda_angle = ['pdda_angle']

In [73]:
params = {
    "test_size":0.33,
    "random_state":42,
    "splits":1,
}

In [74]:
anchor_df = get_data_per_anchor(df,1.0)
X = anchor_df[features].values
y = anchor_df[targets].values
gss = GroupShuffleSplit(n_splits=params["splits"], test_size=params["test_size"], random_state=params["random_state"])
fold = 1
for train_idx, test_idx in gss.split(X, y, groups=anchor_df.point):
    train_idx = shuffle(train_idx, random_state=params['random_state'])
    test_idx = shuffle(test_idx, random_state=params['random_state'])
    train = anchor_df.iloc[train_idx]
    test = anchor_df.iloc[test_idx]
    assert round(test.shape[0]/(train.shape[0]+test.shape[0]), 2) == params["test_size"]
    assert len(set(train.point) & set(test.point)) == 0
    X_train = train[features].values
    y_train = train[targets].values
    X_test = test[features].values
    y_test = test[targets].values
    regr = RandomForestRegressor(random_state=params['random_state'],n_estimators=3)
    regr.fit(X_train,y_train.ravel())
    preds = regr.predict(X_test)
    mae = mean_absolute_error(preds, y_test)
    print(mae)

7.274951750758204


In [75]:
regr

RandomForestRegressor(n_estimators=3, random_state=42)

In [76]:
with open('RandomForestRegressor.h', 'w') as file:
    file.write(port(regr))

In [77]:
inputs = X_test[:20]
preds = preds[:20]
true = y_test[:20]

In [78]:
np.set_printoptions(suppress=True)

In [87]:
np.round(preds, 4)

array([ 17.3333,  41.4667,  -2.9333,  15.1667, -22.1333,  13.8   ,
        43.5667, -20.5   ,  19.5667, -40.9333,  33.4333,  25.9667,
        33.6333,  30.3333,  16.0333,  34.7333, -14.2333, -24.8333,
        36.6667,  24.4667])

In [43]:
true

array([[ 28.9],
       [ 22.5],
       [  8.7],
       [ 42.4],
       [-28.1],
       [ 23.5],
       [  2.7],
       [ -7.8],
       [ -0. ],
       [ -6.8],
       [ 39.7],
       [ 40.8],
       [ 31.5],
       [ 45. ],
       [ 26.6],
       [ 27.8],
       [ 38.2],
       [  0. ],
       [ 14.7],
       [ 31.5]])

In [44]:
inputs

array([[-1.09224092e-01,  3.45719106e-02,  2.96304972e-02,
        -5.72067603e-03,  2.08427706e-02, -1.64787595e-02,
         8.16993320e-03, -4.34533289e-02, -3.56285834e-02,
        -2.15640498e-02],
       [-1.75237932e-01,  3.18694415e-02,  2.30166067e-02,
         3.71943763e-03,  1.33030791e-02,  6.15046235e-03,
         1.20642936e-02,  1.27320587e-02,  2.01076320e-03,
         2.38694833e-02],
       [-5.28740909e-03,  7.83443967e-02,  7.80335219e-02,
        -2.30127868e-02,  7.65018285e-02, -4.72779721e-02,
         9.75279754e-02, -1.93273720e-02,  1.13443881e-01,
        -1.11667373e-02],
       [ 5.79173307e-02,  1.50434723e-01,  1.30507119e-01,
        -6.31178217e-03,  9.80656761e-02, -2.56494866e-02,
         2.37100580e-02, -1.80389979e-01, -1.90593692e-01,
        -4.44590057e-02],
       [ 8.88174257e-02,  2.41465501e-01,  2.30023348e-01,
        -6.28820264e-02,  1.51085220e-01, -1.29114127e-01,
         1.07395320e-01,  1.26339282e-01, -1.04792133e-01,
         9.

In [ ]:
# pdda and models box plot
# pdda in the room and best model
# clean up the code and add to github timon
# mixture of experts